In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import pandas as pd  
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import classification_report
from sklearn import metrics
import numpy as np
from sklearn.decomposition import PCA
from tensorflow.python.keras.models import Sequential
from keras import optimizers
from tensorflow.python.keras.layers import Dense, Activation
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle

Using TensorFlow backend.
C:\Users\Saloni\Anaconda3\envs\my_flask_env\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
data = pd.read_csv('maruti_data.csv')
#print(data)
x=data.iloc[453:2713,6:47]
y=data.iloc[453:2713,4]
print(x)
#print(y)
"""kpca = KernelPCA(kernel="rbf", gamma=10)
X_kpca = kpca.fit_transform(x)
#X_back = kpca.inverse_transform(X_kpca)
pca = PCA(n_components=36)"""
x_std = StandardScaler().fit_transform(x)
pca = PCA(n_components=36)
principalComponents = pca.fit_transform(x_std)
df = pd.DataFrame(data = principalComponents)
df.to_csv('pca.csv')
data = pd.read_csv('pca.csv')
#print(data)

          A/D line         ohlc  aroon up  aroon down         ATR       DMI+  \
453   2.596283e+07  1562.324982        60          30   40.396208  19.991633   
454   2.484155e+07  1539.225006        50          20   41.621482  18.017173   
455   2.335277e+07  1490.612518        40          90   43.134237  16.143488   
456   2.262154e+07  1464.075012        30          90   42.553220  15.195059   
457   2.205769e+07  1444.262512        20          90   42.563709  14.106221   
458   2.224038e+07  1449.250000        10          80   41.255587  13.513962   
459   2.181372e+07  1448.187500         0          70   40.237331  12.866239   
460   2.176941e+07  1431.512512         0          90   40.127518  11.979916   
461   2.140363e+07  1427.024994         0          90   39.564838  11.282413   
462   2.090090e+07  1418.625000         0          90   39.220921  11.315074   
463   2.100855e+07  1423.087494         0          80   39.276570  11.037550   
464   2.033805e+07  1429.350006         

In [7]:
from pandas import Series
from statsmodels.tsa.stattools import adfuller
series = Series.from_csv('pca.csv')
X = series.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -7.757382
p-value: 0.000000
Critical Values:
	1%: -3.433
	5%: -2.863
	10%: -2.567


C:\Users\Saloni\Anaconda3\envs\my_flask_env\lib\site-packages\pandas\core\series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


In [24]:
data = pd.read_csv('g_op10.csv')
df = pd.DataFrame(data)
headers = df.dtypes.index
print(headers)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', 'class'],
      dtype='object')


In [27]:
from sklearn.model_selection import train_test_split
X=data.iloc[:2501,0:34]
Y=data.iloc[:2501,34]
#print(X)
print(Y)

0       1551.449951
1       1515.849976
2       1461.650024
3       1453.300049
4       1433.150024
5       1454.750000
6       1441.000000
7       1428.750000
8       1414.849976
9       1413.449951
10      1429.349976
11      1419.300049
12      1458.349976
13      1452.750000
14      1444.449951
15      1446.199951
16      1369.800049
17      1378.300049
18      1389.449951
19      1398.949951
20      1373.550049
21      1377.250000
22      1365.150024
23      1358.900024
24      1362.050049
25      1352.750000
26      1333.900024
27      1356.349976
28      1336.000000
29      1357.349976
           ...     
2228    7336.250000
2229    7357.950195
2230    7208.399902
2231    7068.850098
2232    7045.399902
2233    7040.600098
2234    6513.399902
2235    6511.049805
2236    6526.950195
2237    6548.250000
2238    6641.149902
2239    6962.299805
2240    6988.200195
2241    7093.600098
2242    7183.399902
2243    7302.549805
2244    7134.250000
2245    7177.899902
2246    7162.500000


In [28]:
ranks = {}
# Create our function which stores the feature rankings to the ranks dictionary
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    print(dict(zip(names, ranks)))
    return dict(zip(names, ranks))
lr = LinearRegression()
lr.fit(X,Y)
#stop the search when only the last feature is left
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(X,Y)
ranks["RFE"] = ranking(list(map(float, rfe.ranking_)), headers, order=-1) 
print ("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), headers)))

{'0': 0.91, '1': 0.06, '2': 0.0, '3': 0.3, '4': 0.61, '5': 0.36, '6': 0.33, '7': 0.09, '8': 0.27, '9': 0.64, '10': 0.73, '11': 0.39, '12': 0.18, '13': 0.52, '14': 0.24, '15': 0.58, '16': 0.7, '17': 0.48, '18': 0.15, '19': 0.21, '20': 0.42, '21': 0.12, '24': 0.67, '25': 0.45, '26': 0.55, '27': 0.82, '28': 0.76, '29': 0.88, '30': 0.79, '31': 0.85, '32': 0.03, '33': 0.94, '34': 1.0, '35': 0.97}
Features sorted by their rank:
[(1, '34'), (2, '35'), (3, '33'), (4, '0'), (5, '29'), (6, '31'), (7, '27'), (8, '30'), (9, '28'), (10, '10'), (11, '16'), (12, '24'), (13, '9'), (14, '4'), (15, '15'), (16, '26'), (17, '13'), (18, '17'), (19, '25'), (20, '20'), (21, '11'), (22, '5'), (23, '6'), (24, '3'), (25, '8'), (26, '14'), (27, '19'), (28, '12'), (29, '18'), (30, '21'), (31, '7'), (32, '1'), (33, '32'), (34, '2')]


In [31]:
from sklearn.model_selection import train_test_split
data = pd.read_csv('g_op10.csv')
train_x=data.iloc[:2247,0:34]
train_y=data.iloc[:2247,34]
test_x=data.iloc[2247:2260,0:34]
test_y=data.iloc[2247:2260,34]
print(test_y)
#train_x, test_x, train_y, test_y = train_test_split(data.iloc[:,1:11], data.iloc[:,12], test_size=0.2, random_state=None, shuffle=False)

2247    7017.500000
2248    6930.049805
2249    6842.700195
2250    6804.549805
2251    6849.899902
2252    6801.549805
2253    6912.399902
2254    6924.899902
2255    6904.299805
2256    6968.500000
2257    6829.700195
Name: class, dtype: float64


In [32]:
def svr_model(train_x, train_y, test_x):
    from sklearn.svm import SVR
    import numpy as np

    clf = SVR(C=1.0, epsilon=2.0, gamma=5e-8)
    clf.fit(train_x, train_y)
    predicted_y = clf.predict(test_x)
    print(predicted_y)
    return predicted_y

In [33]:
predict_svr=svr_model(train_x, train_y, test_x)

[2541.35530204 2541.3549744  2541.35486296 2541.35479704 2541.35479679
 2541.35469905 2541.35471118 2541.35474226 2541.35463426 2541.35468393
 2541.35444527]


In [34]:
def random_forest_reg(train_x, train_y, test_x):
    from sklearn.ensemble import RandomForestRegressor
    import numpy as np
    rfr = RandomForestRegressor(max_depth=5, random_state=0)
    rfr.fit(train_x, train_y)
    rfr_predicted_y = rfr.predict(test_x)
    return rfr_predicted_y


In [35]:
predict_rf=random_forest_reg(train_x, train_y, test_x)


In [36]:
from sklearn.metrics import mean_squared_error
from math import sqrt


def error_function(test_y, predicted_y):
    import numpy as np
    from math import sqrt
    test = list(test_y)
    MAPE=[(test[i]-predicted_y[i])/test[i] for i in range(len(test))]
    MAPE1=abs((sum(MAPE)/len(test)))*100
    MAE=abs((sum(MAPE)/len(test)))
    rRMSE=[((test[i]-predicted_y[i])/test[i])**2 for i in range(len(test))]
    rRMSE=sqrt(sum(rRMSE)/len(test))
    return MAPE1,rRMSE,MAE

In [37]:
def gradient_boost(train_x, train_y, test_x):
    clf = GradientBoostingRegressor(loss='quantile',learning_rate=0.001, n_estimators=20, max_features='log2', min_samples_split=2, max_depth=1)
    clf.fit(train_x, train_y)
    predicted_y = clf.predict(test_x)
    return predicted_y

In [38]:
from sklearn.ensemble import GradientBoostingRegressor 
predict_gr=gradient_boost(train_x, train_y, test_x)

In [18]:
print("\t MAPE\t\tRMSE\t\tMAE")
print("SVR:",error_function(test_y, predict_svr))
print("Random_forest:",error_function(test_y, predict_rf))
print("gradient_boost:",error_function(test_y,predict_gr))

	 MAPE		RMSE		MAE
SVR: (76.93270751804894, 0.769330272095141, 0.7693270751804895)
Random_forest: (3.441501815414504, 0.04705754951740567, 0.03441501815414504)
gradient_boost: (12.750371549453751, 0.12796713295117065, 0.1275037154945375)
